<font size="+3.8">Scanpy single-cell pre-processing</font>  
<font size="+1.5"></font>  

Aim: Preprocess mouse brain single-cell data from Yao 2021 Cell  
Publication: https://pubmed.ncbi.nlm.nih.gov/34004146/

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import matplotlib.pyplot as plt
import os
from datetime import date
from matplotlib.pyplot import rc_context

In [ ]:
sc.settings.verbosity = 3

In [ ]:
main_dir='\\\isdsynnas.srv.med.uni-muenchen.de\BD-Dichgans\SF' # Win

In [ ]:
main_dir='/cluster2/sfrerich/jupyter/p6-foxf2-per-celltype/data' # HPC

# Load data

## Annotated 
Downloaded from: https://portal.brain-map.org/atlases-and-data/rnaseq/mouse-whole-cortex-and-hippocampus-10x (02/2023)

(ucsc h5ad tsv file too large to read in one https://cells.ucsc.edu/?ds=allen-celltypes+mouse-cortex+mouse-cortex-2020)

In [ ]:
# #%%time
# ad = pd.read_hdf(main_dir+"/P6_vasc_scRNAseq/Yao_2021_ABA/aba/expression_matrix.hdf5", key='data')#.transpose()
# # wrong format

In [ ]:
# %%time
# 
# # too largead = sc.read_csv(main_dir+"/P6_vasc_scRNAseq/Yao_2021_ABA/ucsc_cellbrowser_data/exprMatrix.tsv.gz", delimiter="\t") #.transpose()

In [ ]:
import h5py
f = h5py.File(main_dir+"/P6_vasc_scRNAseq/Yao_2021_ABA/aba/expression_matrix.hdf5", 'r')

In [ ]:
f

In [ ]:
list(f.keys())

In [ ]:
dset = f['data']
dset

In [ ]:
list(dset.keys())

In [ ]:
counts = dset['counts']
counts

In [ ]:
counts.shape

In [ ]:
counts.dtype

In [ ]:
counts[0]

In [ ]:
counts # needs transposition

In [ ]:
gene = dset['gene']
gene

In [ ]:
gene.shape

In [ ]:
gene[0:10]

In [ ]:
samples = dset['samples']
samples

In [ ]:
samples.shape

In [ ]:
samples[80000:80003]

In [ ]:
gene = pd.DataFrame(np.array(gene))[0].astype(str).str.extract("b\'(.*)\'")

In [ ]:
samples = pd.DataFrame(np.array(samples))[0].astype(str).str.extract("b\'(.*)\'")

In [ ]:
adata = sc.AnnData(counts, 
        obs=samples, var=gene)

In [ ]:
# load, transpose, and append to adata object in chunks

In [ ]:
adata = AnnData(chunked=True)

# Set the obs and var attributes of the AnnData object
adata.obs = samples
adata.var = gene

# Iterate over the chunks and set the X attribute
for i in range(0, counts.shape[1], 100000):
    chunk = counts[:, i:i+100000]
    adata.chunked.X[i:i+100000] = chunk

In [ ]:
# Create the AnnData object with empty X, obs, and var
adata = AnnData(X=None, obs=samples, var=gene)

# Set the shape of the X attribute based on counts
adata._shape = counts.shape

# Iterate over the chunks and set the X attribute
for i in range(0, counts.shape[1], 100000):
    chunk = counts[:, i:i+100000]
    adata._chunked_X[i:i+100000] = chunk

# Update n_obs and n_vars based on the shape of X
adata.n_obs, adata.n_vars = adata.shape


In [ ]:
adata

In [ ]:
adata = sc.AnnData(counts[:,0:10].T, 
        obs=samples[0:10], var=gene)

In [ ]:
adata

In [ ]:
adata.obs.head()

In [ ]:
pd.set_option('display.max_columns', None)  

In [ ]:
meta = pd.read_csv(main_dir+"/P6_vasc_scRNAseq/Yao_2021_ABA/aba/metadata.csv")
meta.head()

In [ ]:
pd.value_counts(meta.class_label)

In [ ]:
pd.value_counts(meta.subclass_label)

Very few non-neuronal cells!